In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import platform
import seaborn as sns
from sklearn import preprocessing
import re

if platform.system() == 'Darwin':
    rc('font', family = 'AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
    rc('font', family=font_name)

from bs4 import BeautifulSoup
import requests
from datetime import datetime
from datetime import timedelta
import time
from dateutil.relativedelta import relativedelta
from sklearn.linear_model import LinearRegression
from matplotlib import font_manager, rc
font_path = 'C:/Windows/Fonts/NGULIM.TTF'
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

import html
from dateutil import parser

In [4]:
vod_all = pd.read_csv('vod_all.csv', encoding='cp949')

In [5]:
duplicates = vod_all[vod_all.duplicated(subset=['rename'], keep=False)]
print(duplicates)

         name  Summary                          actors  director BigCategory  \
5        극한직업      NaN  류승룡,이하늬,진선규,이동휘,공명,신하균,오정세,김의성       NaN          영화   
29     마카앤로니2      NaN                마카,로니,알버트 박사,알바고       NaN          키즈   
31     마카앤로니2      NaN                마카,로니,알버트 박사,알바고       NaN     TV애니메이션   
55     라바 시즌1      NaN                          옐로우,레드       NaN          키즈   
85    새로운아따맘마      NaN                   엄마,아빠,오아리,오동동       NaN          키즈   
...       ...      ...                             ...       ...         ...   
4568       오싱      NaN                      김민희,안해숙,임혁       NaN          영화   
4615     신데렐라      NaN                     도지원,신세경,유다인       NaN          영화   
4621    가족의탄생      NaN                 문소리,고두심,엄태웅,공효진       NaN          영화   
4642     오아시스      NaN                 문소리,설경구,안내상,박명신       NaN          영화   
4670      황진이      NaN  장미희,안성기,신일룡,전무송,김윤경,조주미,윤양하,박암       NaN          영화   

     SmallCategory Category runningtime

In [10]:
vod_all[vod_all['rename'].str.contains('가족의탄생') ]

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY,rename,counts
749,가족의 탄생,NaN,"이소연,이규한,이채영,김진우",NaN,TV드라마,기타,TV프로그램,36분,가족의 탄생 최종화. 수정은 거래업체로 부터 회장과 성이 다르면서 회장 딸이라 속였...,가족의탄생,1
4621,가족의탄생,NaN,"문소리,고두심,엄태웅,공효진",NaN,영화,드라마,영화,113분,형철은 5년동안 소식없다 불현듯 누나 미라를 찾아온다. 20살 연상녀인 무신과 함께...,가족의탄생,6


In [12]:
vod_all[vod_all['counts']== 0]

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY,rename,counts


In [15]:
def crawling(vod_list):
    for idx, row in vod_list.iterrows():
        # 변수 초기화
        movie_name = ''
        runtime = ''
        daum_director = ''
        director = ''
        summary = ''
        
        a_smry = ''
        a_name = ''
        directors = ''
        # director가 NaN일 때만 채우기
        if pd.isna(row['director']) or row['director'] == '':
            name = row['name']
            category = row['Category']
            print(name)
            print(category)
            # 영화 정보 크롤링
            if category == '영화':
                # 크롤링 검색어준비 - 이름, 러닝타임, 줄거리 - '영화 name' / 감독 - '영화 name 출연진'
                movie_keyword = ' '.join(['영화', name])
                director_keyword = ' '.join([name, '출연진'])
                # url 준비
                movie_url = f"https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={movie_keyword}"
                director_url = f"https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={director_keyword}"
                # 웹페이지 준비
                movie_web_page = requests.get(movie_url).text
                director_web_page = requests.get(director_url).text
                # beautifulsoup 준비
                movie_soup = BeautifulSoup(movie_web_page, 'html.parser')
                director_soup = BeautifulSoup(director_web_page, 'html.parser')
                
                # 파싱 결과 - 영화 이름
                # 못 찾으면 다음에서 검색
                movie_name_html = movie_soup.select('body > div > div > div > div > div > div > div > h2 > span > strong')
                # 네이버에서 못 찾았을 경우 다음에서 검색
                if movie_name_html == []:
                    movie_daum_url = f'https://search.daum.net/search?nil_suggest=btn&w=tot&DA=SBC&q={movie_keyword}'
                    movie_daum_page = requests.get(movie_daum_url).text
                    movie_daum_soup = BeautifulSoup(movie_daum_page, 'html.parser')
                    movie_name_daum_html = movie_daum_soup.select('body > div > div > main > div > div > div > div > div > div > div > div > div > a > b')
                    try: 
                        movie_name = str(movie_name_daum_html).split('>')[1].split('<')[0]
                        print(f'다음 영화 제목 : {html.unescape(movie_name)}')
                    except:
                        print(f'다음 영화 {name} 검색 못함')
                # 네이버에서 찾았을 경우
                else:
                    try:
                        movie_name = str(movie_name_html).split('>')[1].split('<')[0]
                        print(f'네이버 영화 제목 : {html.unescape(movie_name)}')
                    except:
                        print(f'네이버 영화 {name} 검색 못함')
                vod_list.at[idx, 'vod_name'] = html.unescape(movie_name)
                
                # 파싱 결과 - 러닝 타임
                # 러닝 타임 안 나오는 영화는 잘못 검색한 것으로 간주
                runtime_html = movie_soup.select('body > div > div > div > div > div > div > div > div > div > dl > div > dd')
                try:
                    # html parse에서 '분'이 들어간 낱말 가져옴
                    parse_runtime_html = [i for i in str(runtime_html).split('>') if '분' in i][0]
                    runtime = parse_runtime_html.split('<')[0]
                    print(runtime)
                except:
                    # 네이버에 없으면 다음에서 검색
                    movie_runtime_daum_url = f'https://search.daum.net/search?nil_suggest=btn&w=tot&DA=SBC&q={movie_keyword}'
                    movie_runtime_daum_page = requests.get(movie_runtime_daum_url).text
                    movie_runtime_daum_soup = BeautifulSoup(movie_runtime_daum_page, 'html.parser')
                    movie_runtime_daum_html = movie_runtime_daum_soup.select('body > div > div > main > div > div > div > div > div > div > div > div > dl > dd')
                    try:
                        runtime = str(movie_runtime_daum_html[1]).split('<')[-2].split('>')[1].strip()
                        print(runtime)
                        
                        # 다음에서 되면 감독까지는 다음에서 가져옴 - 줄거리는 제공된 데이터 사용
                        daum_director = str(movie_runtime_daum_html[2]).split('</a>')[0].split('>')[-1]
                        vod_list.at[idx, 'director'] = daum_director
                    except:
                        print(f'영화 {name} 러닝타임 없음')
                vod_list.at[idx, 'running_time'] = runtime
                
                time.sleep(5)
                # 파싱 결과 - 감독
                director_html = director_soup.select('body > div > div > div > div > div > div > div > div > div > div > div > div > ul > li')
                try:
                    director = str(director_html).split('<span class="_text">')[1].split('<')[0]
                    # 러닝타임 네이버에 없을 때 다음에서 가져온 감독과 비교 - 대부분 다음이 더 정확해서 다음 감독으로 가져옴
                    if daum_director != '' and director != daum_director:
                        director = daum_director
                    else:
                        print(f'{name} 감독 : {director}')
                except Exception as e:
                    print(e)
                    director_keyword = ' '.join(['영화', html.unescape(movie_name), '출연진'])
                    director_url = f"https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={director_keyword}"
                    director_web_page = requests.get(director_url).text
                    director_soup = BeautifulSoup(director_web_page, 'html.parser')
                    director_html = director_soup.select('body > div > div > div > div > div > div > div > div > div > div > div > div > ul > li')
                    try:
                        director = str(director_html).split('<span class="_text">')[1].split('<')[0]
                        # 러닝타임 네이버에 없을 때 다음에서 가져온 감독과 비교 - 대부분 다음이 더 정확해서 다음 감독으로 가져옴
                        if daum_director != '' and director != daum_director:
                            director = daum_director
                        else:
                            print(f'{name} 감독 : {director}')
                    except Exception as e:
                        print('네이버', e)
                        if daum_director == '':
                            print(f'{name} 감독 못 찾음')
                        else:
                            director = daum_director
                            print(f'다음 {name} 감독 : {daum_director}')
                
                vod_list.at[idx, 'director'] = director
                
                
                # 파싱 결과 - 줄거리
                summary_html = movie_soup.select('body > div > div > div > div > div > div > div > div > div > p')
                if summary_html == []:
                    summary_html = movie_soup.select('body > div > div > div > div > div > div > div > div > div > div > span')
                    if summary_html == []:
                        time.sleep(5)  # 5초 동안 대기
                        movie_keyword = ' '.join([name, '정보'])
                        movie_url = f"https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={movie_keyword}"
                        movie_web_page = requests.get(movie_url).text
                        movie_soup = BeautifulSoup(movie_web_page, 'html.parser')
                        summary_html = movie_soup.select('body > div > div > div > div > div > div > div > div > div > p')
                        if summary_html == []:
                            summary_html = movie_soup.select('body > div > div > div > div > div > div > div > div > div > div > span')
                            if summary_html == []:
                                print(f'영화 {name} 줄거리 못 찾음')
                                time.sleep(5)
                            else:
                                summary = str(summary_html[0]).split(">")[1].split("<")[0]
                                vod_list.at[idx, 'Summary'] = html.unescape(summary)
                                print(html.unescape(summary))
                        else:
                            summary = str(summary_html[0]).split(">")[1].split("<")[0]
                            vod_list.at[idx, 'Summary'] = html.unescape(summary)
                            print(html.unescape(summary))
                    else:
                        summary = str(summary_html[0]).split(">")[1].split("<")[0]
                        vod_list.at[idx, 'Summary'] = html.unescape(summary)
                        print(html.unescape(summary))
                else:
                    summary = str(summary_html[0]).split(">")[1].split("<")[0]
                    vod_list.at[idx, 'Summary'] = html.unescape(summary)
                    print(html.unescape(summary))
            
                time.sleep(5)
            
            # 영화 외 정보 가져오기 - 이름, 줄거리, 연출자
            else:
                # 줄거리
                asset_name = ' '.join([name, '소개'])
                asset_url = f"https://search.daum.net/search?nil_suggest=btn&w=tot&DA=SBC&q={asset_name}"
                asset_web_page = requests.get(asset_url).text
                a_soup = BeautifulSoup(asset_web_page, 'html.parser')
                smry_list = a_soup.select('body > div > div > main > div > div > div > div > div > div > div > div > div > div > dl > dd')
                if smry_list == []:
                    print()
                else:
                    try:
                        a_smry = str(smry_list).split('class="highlighting">')[1].split('<')[0]
                        print(html.unescape(a_smry))
                    except:
                        try:
                            a_smry = str(smry_list[0]).split(">")[1].split("<")[0]
                            print(html.unescape(a_smry))
                        except:
                            print()
                vod_list.at[idx, 'Summary'] = html.unescape(a_smry)
                
                # 이름
                name_list = a_soup.select('body > div > div > main > div > div > div > div > div > div > div > span > a')
                try:
                    a_name = str(name_list).split('<')[-2].split('>')[-1]
                    print(f'{name} 이름 : {html.unescape(a_name)}')
                except:
                    print(f'{name} 이름 못 찾음')
                vod_list.at[idx, 'vod_name'] = html.unescape(a_name)
                
                # 연출자
                asset_name = ' '.join([name, '제작'])
                asset_url = f"https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&sug=&sugo=&sq=&o=&q={asset_name}"
                asset_web_page = requests.get(asset_url).text
                a_soup = BeautifulSoup(asset_web_page, 'html.parser')
                a_list = a_soup.select('body > div > div > main > div > div > div > div > div > div > div > div > div > div > dl > dd')
                if a_list == []:
                    a_list = a_soup.select('body > div > div > main > div > div > div > div > div > div > div > div > div > div > dl > dd > span > span')
                try:
                    infos = str(a_list[1]).split('<span class="highlighting">')[1:]
                    director_list = [info.split('<')[0] for info in infos]
                    directors = ", ".join(director_list)
                    print(directors)
                except:
                    print(f'{name} 연출자 못 찾음')
                vod_list.at[idx, 'director'] = directors

In [18]:
crawling(vod_all)

라바 인 뉴욕
키즈
옐로우와 레드, 두 애벌레의 상상초월 코믹스러운 상황으로 웃음을 자아내는 애니메이션
라바 인 뉴욕 이름 : 라바 인 뉴욕
라바 인 뉴욕 연출자 못 찾음
김과장 특별판
TV프로그램

김과장 특별판 이름 못 찾음
김과장 특별판 연출자 못 찾음
감자아저씨의 크리스마스 공연 외
키즈

감자아저씨의 크리스마스 공연 외 이름 못 찾음
감자아저씨의 크리스마스 공연 외 연출자 못 찾음
밤에 나오는 동물들 외
키즈

밤에 나오는 동물들 외 이름 못 찾음
밤에 나오는 동물들 외 연출자 못 찾음
내가 만든 향수 외
키즈

내가 만든 향수 외 이름 못 찾음
내가 만든 향수 외 연출자 못 찾음
할아버지와 배 타기 외
키즈

할아버지와 배 타기 외 이름 못 찾음
할아버지와 배 타기 외 연출자 못 찾음
수족관에 간 골디 외
키즈

수족관에 간 골디 외 이름 못 찾음
수족관에 간 골디 외 연출자 못 찾음
중세의 성 외
키즈

중세의 성 외 이름 못 찾음
중세의 성 외 연출자 못 찾음
휴양지 숙소 외
키즈

휴양지 숙소 외 이름 못 찾음
휴양지 숙소 외 연출자 못 찾음
강 건너기 외
키즈

강 건너기 외 이름 못 찾음
강 건너기 외 연출자 못 찾음
요란한 가족 외
키즈

요란한 가족 외 이름 못 찾음
요란한 가족 외 연출자 못 찾음
최고의 애완동물 외
키즈

최고의 애완동물 외 이름 못 찾음
최고의 애완동물 외 연출자 못 찾음
마카앤로니2
키즈
천재 발명가 알버트 박사와 그를 따르는 조수 '마카', 그리고 신입 '로니'가 펼치는 코믹한 이야기
마카앤로니2 이름 : 마카앤로니2
마카앤로니2 연출자 못 찾음
마카앤로니2
TV프로그램
천재 발명가 알버트 박사와 그를 따르는 조수 '마카', 그리고 신입 '로니'가 펼치는 코믹한 이야기
마카앤로니2 이름 : 마카앤로니2
마카앤로니2 연출자 못 찾음
짱구는못말려 10기
TV프로그램
떡잎마을에 살고 있는 5살 유치원생 짱구와 말년 계장 아빠, 알뜰한 가정주부 엄마, 그리고 여동생 짱아가 벌이는 소란스러운 짱구 가족의 이야기
짱구는못말

니급아적희환 연출자 못 찾음
나강연화나강우
TV프로그램
음모가 끊이지 않는 궁에서 각종 어려움을 헤쳐나가고 지혜와 용감함으로 분투하며 점차 성장해나가는 이야기
나강연화나강우 이름 : 나강연화나강우
나강연화나강우 연출자 못 찾음
6.15남북공동선언20주년특집그날의증언
TV프로그램

6.15남북공동선언20주년특집그날의증언 이름 못 찾음
6.15남북공동선언20주년특집그날의증언 연출자 못 찾음
설특집동백꽃필무렵다시보기
TV프로그램

설특집동백꽃필무렵다시보기 이름 못 찾음
설특집동백꽃필무렵다시보기 연출자 못 찾음
2023 WWE NXT
TV프로그램

2023 WWE NXT 이름 못 찾음
2023 WWE NXT 연출자 못 찾음
월간 아기상어
키즈

월간 아기상어 이름 못 찾음
월간 아기상어 연출자 못 찾음
고방부자상 적국의 연인
영화
다음 영화 고방부자상 적국의 연인 검색 못함
영화 고방부자상 적국의 연인 러닝타임 없음
list index out of range
네이버 list index out of range
고방부자상 적국의 연인 감독 못 찾음
엇갈린 세력의 남녀 주인공이 서로의 책략으로 겨루고 해치게 되지만 오히려 점점 사랑에 빠지는 무협 로맨스 드라마
90일의 약혼자 시즌6
TV프로그램

90일의 약혼자 시즌6 이름 못 찾음
90일의 약혼자 시즌6 연출자 못 찾음
총몇명TV
TV프로그램

총몇명TV 이름 못 찾음
총몇명TV 연출자 못 찾음
런민기 카트라이더 3
TV프로그램

런민기 카트라이더 3 이름 못 찾음
런민기 카트라이더 3 연출자 못 찾음
라바 키즈동요 Part3
키즈

라바 키즈동요 Part3 이름 못 찾음
라바 키즈동요 Part3 연출자 못 찾음
라바 키즈동요 Part2
키즈

라바 키즈동요 Part2 이름 못 찾음
라바 키즈동요 Part2 연출자 못 찾음
옐언니TV
TV프로그램

옐언니TV 이름 못 찾음
옐언니TV 연출자 못 찾음
I LAND 스페셜 : 새로운 시작
TV프로그램

I LAND 스페셜 : 새로운 시작 이름 못 찾음
I LAND 스페셜 :

목욕하는 풍속점의 새엄마
영화
다음 영화 목욕하는 풍속점의 새엄마 검색 못함
영화 목욕하는 풍속점의 새엄마 러닝타임 없음
list index out of range
네이버 list index out of range
목욕하는 풍속점의 새엄마 감독 못 찾음
영화 목욕하는 풍속점의 새엄마 줄거리 못 찾음
일년만에 폭풍섹스 새엄마
영화
다음 영화 일년만에 폭풍섹스 새엄마 검색 못함
영화 일년만에 폭풍섹스 새엄마 러닝타임 없음
list index out of range
네이버 list index out of range
일년만에 폭풍섹스 새엄마 감독 못 찾음
영화 일년만에 폭풍섹스 새엄마 줄거리 못 찾음
20대 어린 유부녀 모텔 노콘 섹스 영상
영화
다음 영화 20대 어린 유부녀 모텔 노콘 섹스 영상 검색 못함
영화 20대 어린 유부녀 모텔 노콘 섹스 영상 러닝타임 없음
list index out of range
네이버 list index out of range
20대 어린 유부녀 모텔 노콘 섹스 영상 감독 못 찾음
영화 20대 어린 유부녀 모텔 노콘 섹스 영상 줄거리 못 찾음
거품여신 노조미 한도초과 오르가즘
영화
다음 영화 거품여신 노조미 한도초과 오르가즘 검색 못함
영화 거품여신 노조미 한도초과 오르가즘 러닝타임 없음
list index out of range
네이버 list index out of range
거품여신 노조미 한도초과 오르가즘 감독 못 찾음
영화 거품여신 노조미 한도초과 오르가즘 줄거리 못 찾음
50대 이모들 속살이 꽉 찬 진또배기
영화
다음 영화 50대 이모들 속살이 꽉 찬 진또배기 검색 못함
영화 50대 이모들 속살이 꽉 찬 진또배기 러닝타임 없음
list index out of range
네이버 list index out of range
50대 이모들 속살이 꽉 찬 진또배기 감독 못 찾음
영화 50대 이모들 속살이 꽉 찬 진또배기 줄거리 못 찾음
소문난 술집 마담의 특별 서비스
영화
다음 영화 소문난 술집 마담의 특별 서비스 검색 못함
영화 소

여대생 '량미미'는 어느 날 즐겨 보던 사극 속으로 빨려 들어가게 되고 학교 선배이자 인기 아이돌인 '경청'의 상대역을 맡게 된다. 자신이 황제의 총애를 받기는 커녕 3화도 못 가 죽고 마는 황후라는 걸 알게 되고 자신의 운명을 바꾸기 위해 노력하는 이야기
진격적황후1 이름 : 진격적황후 1
진격적황후1 연출자 못 찾음
무림유협기
TV프로그램

무림유협기 이름 못 찾음
무림유협기 연출자 못 찾음
절대쌍교 : 소어아와 화무결
TV프로그램

절대쌍교 : 소어아와 화무결 이름 못 찾음
절대쌍교 : 소어아와 화무결 연출자 못 찾음
꽉잡아
TV프로그램
독특한 세계관을 가진 아이들이 모여있는 특성화고등학교, 기능경기대회를 통해 자신의 꿈과 사랑을 실현해 나아가는 이야기를 그린 애니메이션
꽉잡아 이름 : 꽉 잡아!
꽉잡아 연출자 못 찾음
24시간 박히고 싶은 음란한 유부녀
영화
다음 영화 24시간 박히고 싶은 음란한 유부녀 검색 못함
영화 24시간 박히고 싶은 음란한 유부녀 러닝타임 없음
list index out of range
네이버 list index out of range
24시간 박히고 싶은 음란한 유부녀 감독 못 찾음
영화 24시간 박히고 싶은 음란한 유부녀 줄거리 못 찾음
하다카의 자서전
영화
다음 영화 하다카의 자서전 검색 못함
영화 하다카의 자서전 러닝타임 없음
list index out of range
네이버 list index out of range
하다카의 자서전 감독 못 찾음
영화 하다카의 자서전 줄거리 못 찾음
큰 엉덩이로 사정없이 돌려버리는 아줌마
영화
다음 영화 큰 엉덩이로 사정없이 돌려버리는 아줌마 검색 못함
영화 큰 엉덩이로 사정없이 돌려버리는 아줌마 러닝타임 없음
list index out of range
네이버 list index out of range
큰 엉덩이로 사정없이 돌려버리는 아줌마 감독 못 찾음
영화 큰 엉덩이로 사정없이 돌려버리는 아줌마 줄거리 못 찾음
경이로운 소문   경이로운 몰아보기
TV프로그램

경이로운 소문   경이

호기심과 상상력 넘치는 5살 천방지축 꼬마소녀 콩순이와 친구들이 놀이를 통해 스스로 깨닫고 성장하는 모습을 그린 프로그램
엉뚱발랄 콩순이와 친구들 5기 이름 : 엉뚱발랄 콩순이와 친구들 5
엉뚱발랄 콩순이와 친구들 5기 연출자 못 찾음
장애인의 날 특집 봄날의 기적
TV프로그램
저소득가정 장애인들이 적기에 치료를 받을 수 있도록 후원을 통해 도울 수 있는 프로그램
장애인의 날 특집 봄날의 기적 이름 : 봄날의 기적
장애인의 날 특집 봄날의 기적 연출자 못 찾음
말량n홍챠TV 시즌2
TV프로그램

말량n홍챠TV 시즌2 이름 못 찾음
말량n홍챠TV 시즌2 연출자 못 찾음
원피스 21기 part5
TV프로그램

원피스 21기 part5 이름 못 찾음
원피스 21기 part5 연출자 못 찾음
부군청자중 : 부군님, 자중하시어요
TV프로그램
작가 목소석은 우연히 자신의 소설 속으로 들어가게 되고 10개의 미션을 달성해야만 현실 세계로 돌아올 수 있는 이야기를 그린 드라마
부군청자중 : 부군님, 자중하시어요 이름 : 부군청자중 : 부군님, 자중하시어요
부군청자중 : 부군님, 자중하시어요 연출자 못 찾음
히스토리 5
TV프로그램

히스토리 5 이름 못 찾음
히스토리 5 연출자 못 찾음
유희왕 듀얼몬스터 Part3
TV프로그램

유희왕 듀얼몬스터 Part3 이름 못 찾음
유희왕 듀얼몬스터 Part3 연출자 못 찾음
차금
TV프로그램

차금 이름 못 찾음
차금 연출자 못 찾음
백앤아 Part2
TV프로그램

백앤아 Part2 이름 못 찾음
백앤아 Part2 연출자 못 찾음
오후의 정사
영화
네이버 영화 제목 : 오후의 정사
64분
오후의 정사 감독 : 니시무라 쇼고로
단조로운 일상과 무뚝뚝한 남편에게 불만을 가지고 있던 ‘리츠코’. 오랜만에 만난 친구 ‘키리무라’와의 술자리에서 취한 그녀는 함께 호텔까지 가게 되고, 남편에게서는 느껴보지 못한 섹스의 쾌감을 느낀다. 다음 날, ‘리츠코’는 둘을 목격한 옆집 이웃 ‘요코’에게 매일 다른 남자와 잘 수 있다며 매춘을 제안 받고, 

...

석조저택 살인사건
영화
네이버 영화 제목 : 석조저택 살인사건
109분
석조저택 살인사건 감독 : 정식
해방 후 경성, 거대한 석조저택에서 두 남자가 마주한다. 그리고 울린 여섯 발의 총성. 최초 신고자의 전화를 받고 경찰이 출동하고, 운전수 ‘최승만’을 살해한 혐의로 경성 최고의 재력가 ‘남도진’이 체포된다. 하지만 현장에 남은 건 사체를 태운 흔적과 핏자국, 그리고 잘려나간 손가락 뿐. 미스터리한 석조저택 살인사건을 두고 치열한 법정공방이 벌어지는데… 정체불명의 운전수 최승만(고수) 경성 최고의 재력가 남도진(김주혁) 사건을 무마하려는 변호사(문성근) 유죄를 입증하려는 검사(박성웅) 치밀하게 계획되고 잔인하게 실행된 살인사건! 이 모든 것은 누군가가 설계한 속임수다!
태백산맥
영화
네이버 영화 제목 : 태백산맥
168분
list index out of range
태백산맥 감독 : 임권택
해방후, 좌우익의 대결이 심화되는 파란의 역사속에서 48년 10월 여순 사건이 터진다. 전라남도 보성군 당 위원장인 염상진을 중심으로 한 좌익들은 벌교를 장악하고 인민 재판을 열어 반동 숙청을 한다. 하지만 반란군 주력의 패퇴로 조계산으로 후퇴하고 만다. 벌교로 돌아온 경찰 등 우익세력들은 좌익 부역자와 가족들을 연행하여 조사하고 대동청년단 감찰부장인 염상구는 형 염상진에 대한 증오심으로 이 일에 앞장선다. 수도경찰 주도로 손가락 재판이 벌어져 무고한 희생자가 발생하는 가운데 염상구는 빨치산 강동식의 아내 외서댁을 겁탈하고 반란 때 처형된 유지의 자식들은 멸공단이라는 단체를 만들어 좌익 가족에 대한 테러를 자행한다. 한편, 전라남도 도당 직속의 정하섭은 무당 소화의 집에 잠입하고, 소화는 그의 심부름을 해주면서 둘 사이에는 신분을 초월하는 사랑이 싹튼다. 순천 중학교의 교사이며 민족주의자인 김범우는 벌교 내에서 벌어진 좌익의 잔인한 반동숙청과 우익의 과도한 보복 양쪽 모두를 비판하고 막아보려고 애를 써보지만 빨갱이로 몰리는 수모를 당하고 멸공단에게 테러를 당하고 만다. 산속

로스트 블레이드
영화
네이버 영화 제목 : 로스트 블레이드
87분
로스트 블레이드 감독 : 손호의
현상금 사냥꾼 귀검이 고향으로 돌아가기 위해 밀수 단주 금만당을 없애는 일에 동조한다. 하지만 귀검의 부인을 탐내던 동료는 그에게 누명을 씌워 유배를 보낸다. 그로부터 10년 뒤, 길 잃은 칼, 귀검은 피의 복수를 선언하는데......
성스러운 아이돌
TV프로그램
하루 아침에 무명 아이돌이 된, 이 세계 대신관 램브러리의 성스럽고 망측한 연예계 적응기
성스러운 아이돌 이름 : 성스러운 아이돌
박소연, 이소윤, 이천금
신데렐라
영화
네이버 영화 제목 : 신데렐라: 트랩 오브 허
100분
list index out of range
신데렐라 감독 : 이안 소프틀리
집에서 일어난 화재 사고로 인해 대대적인 성형수술을 한 ‘미키’(튜펜스 미들턴)는 과거에 대해 아무것도 기억하지 못한 채 깨어난다. 자신이 누군지, 어떤 사람이었는지 알기 위해 미키는 관련된 사람들을 찾아나서고, 그러던 중 그녀의 옛 애인에게서 자신의 가장 친한 친구였던 ‘도’(알렉산드라 로치)의 죽음에 대해 전해 듣는다. 미키는 도의 집에 찾아가고, 그 곳에서 도가 썼던 일기장을 읽게 된다. 누구에게나 사랑받았던 여자 미키와 그런 미키를 동경했던 도. 미키를 좋아하고, 미키를 닮고자 하는 것을 넘어 자기 자신이 미키가 되고 싶었던 도의 계획이 점점 드러나는데...
마검생사기
TV프로그램
복잡한 무림 속에서 일련의 권력 싸움과 피비린내 나는 바람이 곧 강호를 휩쓸게 되고, 우여곡절 한 사랑의 전설과 파란만장한 무림의 기연을 그리게 되는 이야기
마검생사기 이름 : 마검생사기
마검생사기 연출자 못 찾음
극장판 귀멸의 칼날: 무한열차편
영화
다음 영화 제목 : 극장판 귀멸의 칼날: 무한열차편
117분 (재)
극장판 귀멸의 칼날: 무한열차편 감독 : 소토자키 하루오
혈귀로 변한 여동생 ‘네즈코’를 되돌리기 위해 귀살대가 된 ‘탄지로’! 어둠 속을 달리는 무한열차에서 승객들이 흔적 없이 사라진다는 소식에 ‘젠이츠’, 

만해 한용운 님의침묵
영화
네이버 영화 제목 : 만해 한용운 님의침묵
144분
만해 한용운 님의침묵 감독 : 한명구
홍주성(지금의 홍성)에서 동학란(동학농민운동)이 일어난다. 16세의 몸으로 동학란에 참여를 하는 소년 만해, 1천냥을 탈취하여 군자금으로 보내는 활동을 한다. 55세 성북동에 있는 심우장에서 만해는 재혼(유숙원)을 한다. 아내의 삯바느질 등으로 생활을 영위하면서도 만해는 집필과 함께 창씨 개명 운동 반대, 조선인 학병 출정 반대 운동 등에 참여를 하며 일제에 항거를 계속 한다. 용정명월 백화마담에게서 군자금을 넘겨 받는다. 이화영이 군자금을 만해와 함께 건낸다. 1944년 만해는 66세를 일기로 입적하고 만다. “님의 침묵” 시가 흐르며 만해의 업적이 하이라이트로 소개되면서 자막이 흐른다.
환장
영화
네이버 영화 제목 : 환장
85분
list index out of range
환장 감독 : 맥당걸
홍콩 나이트클럽에서 일하는 아가씨들을 노리는 연쇄 살인 사건이 일어나자 홍콩 경찰은 문제아 여성 경찰 일곱 명을 클럽 아가씨로 위장해 홍콩에서 가장 큰 드래곤 나이트클럽에 잠입시킨다. 하지만 일곱 여경은 자신들의 목적을 잊어버리고 클럽에서의 아가씨 팀끼리의 세력 다툼에 더욱 몰두하게 되는데…
소태극
영화
네이버 영화 제목 : 소태극

소태극 감독 : 원화평
구두쇠로 악명높은 아버지밑에 초와 단 두형제가 있는데 형 초는 착실하게 아버지 밑에서 일을 하고 동생 단은 장난으로 하루하루를 보내는 건달이다. 그러던중 아버지의 옛날 원수가 아버지와 형을 찾아오고 그시간 단은 마을 주점에서 돈내기 힘겨루기를 하고 아침에 집에가니 아버지와 형은 살해되고 집은 잿더미가 되어있고 단마저 생명의 위협을 받던중 태극취권의 대가인 사부 비를 만나 힘들고 고된 고도의 무술을 연마하여 드디어 원수와 한판을 겨루게 되는데.... (spider1983님)
조폭 마누라 3
영화
네이버 영화 제목 : 조폭 마누라 3
115분
조폭 마누라 3 감독 : 조진규
홍콩 최고의 명문 조직 화백련

오타쿠 엘프
TV프로그램
400년의 역사를 간직한 신사에 모셔진 이세계로부터 온 엘프와 소녀의 일상을 그린 코미디 애니메이션
오타쿠 엘프 이름 : 오타쿠 엘프
안자이 타케후미
돌아온 판관 포청천   타룡포
영화
네이버 영화 제목 : 돌아온 판관 포청천 - 타룡포
90분
돌아온 판관 포청천   타룡포 감독 : 유립립
황제 출생의 비밀 이야기. 어느 노부인이 포증에게 자신은 전 황제의 후궁이며 현 황제는 자신의 아들이라는 사실을 밝히게 되는데…
황제의 검
영화
네이버 영화 제목 : 황제의 검
87분
list index out of range
황제의 검 감독 : 교뢰
'황제의 검'이 되어 탐관오리를 숙청하던 천하제일 검객이 그 자리에 환멸을 느껴 모습을 감춘다. 10년 뒤, 그를 이어 '황제의 검'이 된 사마치가 간신모리배와 결탁해 백성을 수탈하는 것을 목격하고… 검객은 정의를 위해 다시 검을 잡는다.
황진이
영화
네이버 영화 제목 : 황진이
141분
list index out of range
황진이 감독 : 장윤현
여자는 땅, 천민은 짐승이던 16세기. 양반가의 딸로 자란 진이(송혜교)는 출생의 비밀이 밝혀지자 가장 천한 ‘기생’의 신분을 스스로 선택한다. 인간으로서 가장 밑바닥으로 추락했으나 사대부조차 동경하는 최고의 여인이 된 ‘진이’. 그녀 곁에 벗이었고, 노비였으며, 첫 남자인 놈이(유지태)가 있다. 시대의 격랑 속에서 놈이는 반역자로 수배되고, 이제 진이는 자신의 전부를 건 운명의 선택을 한다.
바빌론
영화
네이버 영화 제목 : 바빌론
189분
바빌론 감독 : 데이미언 셔젤
황홀하면서도 위태로운 고대 도시, '바빌론'에 비유되던 할리우드. '꿈' 하나만을 위해 모인 사람들이 이를 쟁취하기 위해 벌이는 강렬하면서도 매혹적인 이야기
라라랜드
영화
네이버 영화 제목 : 라라랜드
127분
라라랜드 감독 : 데이미언 셔젤
꿈을 꾸는 사람들을 위한 별들의 도시 ‘라라랜드’. 재즈 피아니스트 ‘세바스찬’(라이언 고슬링)과 배우 지망생 ‘미아’(엠마 스톤), 인생에서 

미지왕 감독 : 김용태
희대의 바람둥이 왕창한과 열살 연상의 재벌 외동딸 엄청난의 결혼식 날. 조촐히 치루기로 합의한 결혼식에 양가 친지를 비롯한 60여명의 하객이 자리를 잡고 신랑의 입장을 기다리는 순간, 결혼식의 주인공 왕창한이 나타나질 않는다. 조금 전까지도 하객들과 인사를 나우던 신랑이 갑자기 사라져 버린 것이다. 마침 하객으로 온 두 경찰과 양가 가족의 협조로 신랑을 찾기 위한 긴급수사 본부가 설치되고 두 경찰은 신랑의 주변 인물들을 차례로 불러 신랑의 실종에 대한 수사를 전개한다. 증인들의 엇갈린 증언 속에서 왕창한의 성장배경과 여성편련등이 조금씩 드러나는 가운데에도 신랑의 실종은 자발적 도피인지 타인의 의한 납치인지 조차도 갈피가 잡히질 않는데. 하객들도 지쳐가고 신랑의 행방도 더욱 오리무중.
꾼
영화
네이버 영화 제목 : 꾼
117분
꾼 감독 : 장창원
대한민국을 발칵 뒤집어 놓은 ‘희대의 사기꾼’ 장두칠이 돌연 사망했다는 뉴스가 발표된다. 그러나 그가 아직 살아있다는 소문과 함께 그를 비호했던 권력자들이 의도적으로 풀어준 거라는 추측이 나돌기 시작한다. 사기꾼만 골라 속이는 사기꾼 지성(현빈)은 장두칠이 아직 살아있다며 사건 담당 검사 박희수(유지태)에게 그를 확실하게 잡자는 제안을 한다. 박검사의 비공식 수사 루트인 사기꾼 3인방 고석동(배성우), 춘자(나나), 김 과장(안세하)까지 합류시켜 잠적한 장두칠의 심복 곽승건(박성웅)에게 접근하기 위한 새로운 판을 짜기 시작한다. 하지만 박검사는 장두칠 검거가 아닌 또 다른 목적을 위해 은밀히 작전을 세우고, 이를 눈치 챈 지성과 다른 꾼들도 서로 속지 않기 위해 각자만의 계획을 세우기 시작하는데… 이 판에선 누구도 믿지 마라! 진짜 ‘꾼’들의 예측불가 팀플레이가 시작된다!
어니스트 씨프
영화
네이버 영화 제목 : 어니스트 씨프
98분
어니스트 씨프 감독 : 마크 윌리엄스
희대의 폭파 전문 은행털이범 ‘톰’. 어떠한 증거도 남기지 않는 뛰어난 범행으로 8년간 FBI의 추적을 따돌린 그는 새로운 인생

In [17]:
# vod_all.to_csv('vod_all_pre.csv', encoding='utf-8')

In [19]:
vod_all.head()

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY,rename,counts,vod_name,running_time
0,와이낫크루 뜻밖의 여행,가는 길이 좀 멀고 험해도 나만의 취저(취향 저격)인 곳은 집요하게 찾아가는 MZ를...,"KCM,권은비,조정식,박현규","은지향, 김한진, 김남훈, 명민아, 김주희",TV 연예/오락,기타,TV프로그램,69분,"# 오늘, 이 동네는 우리가 알잘딱깔센 접수한다! 뻔한 랜드마크는 궁금하지도 않고,...",와이낫크루뜻밖의여행,16,와이낫크루: 뜻밖의 여행,NaN
1,그레이트 뷰티,40여 년 전 소설 한 권을 끝으로 더 이상 책을 쓰지 못하는 젭은 로마 1%의 삶...,"토니 세르빌로,사브리나 페릴리,세레나 그랜디,이사벨라 페라리",파올로 소렌티노,영화,기타,영화,0,(HD)그레이트 뷰티,그레이트뷰티,1,그레이트 뷰티,141분
2,해안선,평온해 보이는 동해안의 바닷가. “경고! 밤 7시 이후 이곳을 접근하는 자는 간첩으...,"장동건,김정학,박지아,유해진",김기덕,영화,액션/어드벤쳐,영화,0,(SD)해안선,해안선,1,해안선,94분
3,아는 형님,이성 상실 본능 충실 형님학교에서 벌어지는 세상의 모든 놀이 [아는 형님],"강호동,서장훈,김영철,이수근,김희철,민경훈,이상민","민철기, 최창수, 이수아, 황선영, 지은정",TV 연예/오락,기타,TV프로그램,87분,"(구)동구 윤시윤, (구)호동 이호원 등장! 아형군 탄생리 이장 선발전! 후보 1번...",아는형님,44,아는 형님,NaN
4,베놈,진실을 위해서라면 몸을 사리지 않는 정의로운 열혈 기자 '에디 브록' 거대 기업 라...,"톰 하디,미셀 윌리엄스,리즈 아메드,제니 슬레이트,레이드 스콧,스콧 헤이즈,우디 해...",루벤 플레셔,영화,액션/어드벤쳐,영화,140분,(본편+부가영상) 서비스 해지 시까지 시청 가능. 좋은 놈 에디브룩(톰 하디) vs...,베놈,2,베놈,107분


In [21]:
column_names = vod_all.columns
column_names
vod_all = vod_all.drop([i for i in column_names if 'Unnamed' in i], axis=1)

In [35]:
# Summary 채우기
vod_all['Summary'] = vod_all['Summary'].fillna(vod_all['SMRY'])
vod_all.loc[vod_all['Summary'] == '', 'Summary'] = vod_all['SMRY']

# director 채우기
vod_all['director'] = vod_all['director'].fillna('-')
vod_all.loc[vod_all['director'] == '', 'director'] = '-'

# running_time 채우기
vod_all['running_time'] = vod_all['running_time'].fillna(vod_all['runningtime'])
vod_all.loc[vod_all['running_time'] == '', 'running_time'] = vod_all['runningtime']
vod_all.loc[vod_all['running_time'] == '0', 'running_time'] = '-'

# vod_name 채우기
vod_all['vod_name'] = vod_all['vod_name'].fillna(vod_all['name'])
vod_all.loc[vod_all['vod_name'] == '', 'vod_name'] = vod_all['name']


In [36]:
# NaN 확인
vod_all.isna().sum()

name             0
Summary          0
actors           0
director         0
BigCategory      0
SmallCategory    0
Category         0
runningtime      0
SMRY             0
rename           0
counts           0
vod_name         0
running_time     0
dtype: int64

In [48]:
# 필요없는 column 삭제 - SMRY, rummingtume
vod_all.drop(['SMRY', 'runningtime'], axis=1, inplace=True)

In [49]:
vod_all

,name,Summary,actors,director,BigCategory,SmallCategory,Category,rename,counts,vod_name,running_time
0,와이낫크루 뜻밖의 여행,가는 길이 좀 멀고 험해도 나만의 취저(취향 저격)인 곳은 집요하게 찾아가는 MZ를...,"KCM,권은비,조정식,박현규","은지향, 김한진, 김남훈, 명민아, 김주희",TV 연예/오락,기타,TV프로그램,와이낫크루뜻밖의여행,16,와이낫크루: 뜻밖의 여행,69분
1,그레이트 뷰티,40여 년 전 소설 한 권을 끝으로 더 이상 책을 쓰지 못하는 젭은 로마 1%의 삶...,"토니 세르빌로,사브리나 페릴리,세레나 그랜디,이사벨라 페라리",파올로 소렌티노,영화,기타,영화,그레이트뷰티,1,그레이트 뷰티,141분
2,해안선,평온해 보이는 동해안의 바닷가. “경고! 밤 7시 이후 이곳을 접근하는 자는 간첩으...,"장동건,김정학,박지아,유해진",김기덕,영화,액션/어드벤쳐,영화,해안선,1,해안선,94분
3,아는 형님,이성 상실 본능 충실 형님학교에서 벌어지는 세상의 모든 놀이 [아는 형님],"강호동,서장훈,김영철,이수근,김희철,민경훈,이상민","민철기, 최창수, 이수아, 황선영, 지은정",TV 연예/오락,기타,TV프로그램,아는형님,44,아는 형님,87분
4,베놈,진실을 위해서라면 몸을 사리지 않는 정의로운 열혈 기자 '에디 브록' 거대 기업 라...,"톰 하디,미셀 윌리엄스,리즈 아메드,제니 슬레이트,레이드 스콧,스콧 헤이즈,우디 해...",루벤 플레셔,영화,액션/어드벤쳐,영화,베놈,2,베놈,107분
...,...,...,...,...,...,...,...,...,...,...,...
4710,베놈 2: 렛 데어 비 카니지,‘베놈’과 완벽한 파트너가 된 ‘에디 브록’(톰 하디) 앞에 ‘클리터스 캐서디’(우...,"톰 하디,미셀 윌리엄스,나오미 해리스,레이드 스콧,스티븐 그레이엄,우디 해럴슨",앤디 서키스,영화,액션/어드벤쳐,영화,베놈2:렛데어비카니지,11,베놈 2: 렛 데어 비 카니지,97분
4711,성월동화,야쿠자 두목 사토는 대도 당걸에게 자신의 일기를 찾도록 의뢰한다. 그러나 당걸에게 ...,"장국영,토키와 타카코,양자경,이찬삼,고첩,혜천사,요계지,황가락",진가상,영화,멜로,영화,성월동화,3,성월동화 2 - 연전충승,98분
4712,다큐인사이트 코드블루,히포크라테스 선서를 외치고 의사들이 출발선에 서는 3월. 전남대병원에 발 디딘 새내...,유승호,-,TV 시사/교양,기타,TV프로그램,다큐인사이트코드블루,1,다큐인사이트 코드블루,47분
4713,캠프 제로베이스원,9인만의 스페셜 에디션 캠핑카를 타고 떠나는 예능 프로그램,제로베이스원,서예진,TV 연예/오락,기타,TV프로그램,캠프제로베이스원,1,캠프 제로베이스원,109분


In [25]:
vod_all.to_csv('vod_list_addinfo.csv', encoding='utf-8')

In [50]:
vod_id = pd.read_csv('vod_all.csv', encoding='cp949')

In [51]:
vod_id

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY,rename,counts,vod_id
0,와이낫크루 뜻밖의 여행,NaN,"KCM,권은비,조정식,박현규",NaN,TV 연예/오락,기타,TV프로그램,69분,"# 오늘, 이 동네는 우리가 알잘딱깔센 접수한다! 뻔한 랜드마크는 궁금하지도 않고,...",와이낫크루뜻밖의여행,16,0
1,그레이트 뷰티,NaN,"토니 세르빌로,사브리나 페릴리,세레나 그랜디,이사벨라 페라리",NaN,영화,기타,영화,0,(HD)그레이트 뷰티,그레이트뷰티,1,1
2,해안선,NaN,"장동건,김정학,박지아,유해진",NaN,영화,액션/어드벤쳐,영화,0,(SD)해안선,해안선,1,2
3,아는 형님,NaN,"강호동,서장훈,김영철,이수근,김희철,민경훈,이상민",NaN,TV 연예/오락,기타,TV프로그램,87분,"(구)동구 윤시윤, (구)호동 이호원 등장! 아형군 탄생리 이장 선발전! 후보 1번...",아는형님,44,3
4,베놈,NaN,"톰 하디,미셀 윌리엄스,리즈 아메드,제니 슬레이트,레이드 스콧,스콧 헤이즈,우디 해...",NaN,영화,액션/어드벤쳐,영화,140분,(본편+부가영상) 서비스 해지 시까지 시청 가능. 좋은 놈 에디브룩(톰 하디) vs...,베놈,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...
4710,베놈 2: 렛 데어 비 카니지,NaN,"톰 하디,미셀 윌리엄스,나오미 해리스,레이드 스콧,스티븐 그레이엄,우디 해럴슨",NaN,영화,액션/어드벤쳐,영화,2분,히어로의 시대는 끝났다. 미워할 수 없는 빌런 히어로 베놈과 완벽한 파트너가 된 에...,베놈2:렛데어비카니지,11,4710
4711,성월동화,NaN,"장국영,토키와 타카코,양자경,이찬삼,고첩,혜천사,요계지,황가락",NaN,영화,멜로,영화,0,히토미는 타츠야와 결혼을 앞두고 홍콩에서의 생활을 위해 광동어를 배우고 있다. 어느...,성월동화,3,4711
4712,다큐인사이트 코드블루,NaN,유승호,NaN,TV 시사/교양,기타,TV프로그램,47분,히포크라테스 선서를 외치고 의사들이 출발선에 서는 3월. 전남대병원에 발 디딘 새내...,다큐인사이트코드블루,1,4712
4713,캠프 제로베이스원,NaN,제로베이스원,NaN,TV 연예/오락,기타,TV프로그램,109분,힐링 끝! 미션 시작! 마지막까지 호락호락하지 않은 제주도에서의 로즈 미션!,캠프제로베이스원,1,4713


In [53]:
vod_add_info = pd.merge(vod_all, vod_id[['rename', 'Category', 'vod_id']], on=['rename', 'Category'], how='left')
vod_add_info

,name,Summary,actors,director,BigCategory,SmallCategory,Category,rename,counts,vod_name,running_time,vod_id
0,와이낫크루 뜻밖의 여행,가는 길이 좀 멀고 험해도 나만의 취저(취향 저격)인 곳은 집요하게 찾아가는 MZ를...,"KCM,권은비,조정식,박현규","은지향, 김한진, 김남훈, 명민아, 김주희",TV 연예/오락,기타,TV프로그램,와이낫크루뜻밖의여행,16,와이낫크루: 뜻밖의 여행,69분,0
1,그레이트 뷰티,40여 년 전 소설 한 권을 끝으로 더 이상 책을 쓰지 못하는 젭은 로마 1%의 삶...,"토니 세르빌로,사브리나 페릴리,세레나 그랜디,이사벨라 페라리",파올로 소렌티노,영화,기타,영화,그레이트뷰티,1,그레이트 뷰티,141분,1
2,해안선,평온해 보이는 동해안의 바닷가. “경고! 밤 7시 이후 이곳을 접근하는 자는 간첩으...,"장동건,김정학,박지아,유해진",김기덕,영화,액션/어드벤쳐,영화,해안선,1,해안선,94분,2
3,아는 형님,이성 상실 본능 충실 형님학교에서 벌어지는 세상의 모든 놀이 [아는 형님],"강호동,서장훈,김영철,이수근,김희철,민경훈,이상민","민철기, 최창수, 이수아, 황선영, 지은정",TV 연예/오락,기타,TV프로그램,아는형님,44,아는 형님,87분,3
4,베놈,진실을 위해서라면 몸을 사리지 않는 정의로운 열혈 기자 '에디 브록' 거대 기업 라...,"톰 하디,미셀 윌리엄스,리즈 아메드,제니 슬레이트,레이드 스콧,스콧 헤이즈,우디 해...",루벤 플레셔,영화,액션/어드벤쳐,영화,베놈,2,베놈,107분,4
...,...,...,...,...,...,...,...,...,...,...,...,...
4710,베놈 2: 렛 데어 비 카니지,‘베놈’과 완벽한 파트너가 된 ‘에디 브록’(톰 하디) 앞에 ‘클리터스 캐서디’(우...,"톰 하디,미셀 윌리엄스,나오미 해리스,레이드 스콧,스티븐 그레이엄,우디 해럴슨",앤디 서키스,영화,액션/어드벤쳐,영화,베놈2:렛데어비카니지,11,베놈 2: 렛 데어 비 카니지,97분,4710
4711,성월동화,야쿠자 두목 사토는 대도 당걸에게 자신의 일기를 찾도록 의뢰한다. 그러나 당걸에게 ...,"장국영,토키와 타카코,양자경,이찬삼,고첩,혜천사,요계지,황가락",진가상,영화,멜로,영화,성월동화,3,성월동화 2 - 연전충승,98분,4711
4712,다큐인사이트 코드블루,히포크라테스 선서를 외치고 의사들이 출발선에 서는 3월. 전남대병원에 발 디딘 새내...,유승호,-,TV 시사/교양,기타,TV프로그램,다큐인사이트코드블루,1,다큐인사이트 코드블루,47분,4712
4713,캠프 제로베이스원,9인만의 스페셜 에디션 캠핑카를 타고 떠나는 예능 프로그램,제로베이스원,서예진,TV 연예/오락,기타,TV프로그램,캠프제로베이스원,1,캠프 제로베이스원,109분,4713


In [54]:
vod_add_info.isna().sum()

name             0
Summary          0
actors           0
director         0
BigCategory      0
SmallCategory    0
Category         0
rename           0
counts           0
vod_name         0
running_time     0
vod_id           0
dtype: int64

In [56]:
vod_add_info.to_csv('vod_add_id.csv', encoding='utf-8')